In [21]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [23]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [24]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [25]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
tx_augmented = build_poly_all_features(tx,4)
y,tx = build_model_data(tx_augmented,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [26]:
num_samples, num_features
tx.shape

(250000, 93)

In [27]:
from compute_gradient import *
from cost import *
from implementations import *

In [28]:
#Initial weight


#Maximum iterations through the whole set
max_iter = 100

#0.001 nan
#0.0005 nan

#Step-size
gammas = np.logspace(-6,-1, 7)
lambdas = np.logspace (-6,-1,7)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 93), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [29]:
losses_tr = np.zeros((len(gammas),len(lambdas)))
losses_val = np.zeros((len(gammas),len(lambdas)))

k_fold = 2
seed = 1
loss_min = np.inf
initial_w = np.zeros((tx.shape[1],1))
max_accuracy = 0
#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    for ind_gamma ,gamma_ in enumerate (gammas):
        loss_training = 0
        loss_validation = 0
        training_accuracy = 0
        validation_accuracy = 0
        for i,k in enumerate(range(k_fold)):
            
            tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
            w, loss_train_k = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma_)
            loss_val_k = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
            #loss_train_k /= len(y_train)
            loss_training += loss_train_k
            loss_validation += loss_val_k
            training_accuracy += predict_accuracy(y_train,tx_train,w)
            validation_accuracy += predict_accuracy(y_val,tx_val,w)
        
        training_accuracy /= k_fold
        validation_accuracy /= k_fold
        loss_training /= k_fold
        loss_validation /= k_fold
        print(lambda_)
        print(gamma_)
        print(loss_training)
        print(loss_validation)
        print(training_accuracy)
        print(validation_accuracy)
        if (validation_accuracy > max_accuracy):
            weights_star = w
            lambda_star = lambda_
            gamma_star = gamma_
            loss_min = loss_validation
            max_accuracy = validation_accuracy
        losses_tr[ind_lambda][ind_gamma] = loss_training
        losses_val[ind_lambda][ind_gamma] = loss_validation



/Users/Sofia/Desktop/ML_Project1/cost.py:20: RuntimeWarning: divide by zero encountered in log
  loss = y.T.dot(np.log(pred)) + (1 - y).T.dot(np.log(1 - pred))


1e-06
1e-06
0.6932169714781314
nan
65.756
65.748
1e-06
6.8129206905796085e-06
0.6910933669324504
nan
65.85319999999999
65.8488
1e-06
4.641588833612782e-05
0.6677891683224545
nan
65.7252
65.73519999999999
1e-06
0.00031622776601683794
0.5794878637267501
nan
64.0036
64.01599999999999
1e-06
0.0021544346900318843
1.3473835458136556
nan
68.1908
68.2644
1e-06
0.014677992676220705
1.0856930103554403
nan
67.7732
67.6896
1e-06
0.1
3.1037406824871216
nan
66.3116
66.2644
6.8129206905796085e-06
1e-06
4.039081737444631
nan
65.9324
65.9324
6.8129206905796085e-06
6.8129206905796085e-06
4.040523684172748
nan
65.9348
65.934
6.8129206905796085e-06
4.641588833612782e-05
4.050338944069868
nan
65.94399999999999
65.94200000000001
6.8129206905796085e-06
0.00031622776601683794
4.116829055010169
nan
65.982
65.9768
6.8129206905796085e-06
0.0021544346900318843
4.554307020107314
nan
66.2348
66.19560000000001
6.8129206905796085e-06
0.014677992676220705
4.083029023472529
nan
68.2492
68.22800000000001
6.8129206905796

In [30]:
lambdas

array([1.00000000e-06, 6.81292069e-06, 4.64158883e-05, 3.16227766e-04,
       2.15443469e-03, 1.46779927e-02, 1.00000000e-01])

In [31]:
gammas

array([1.00000000e-06, 6.81292069e-06, 4.64158883e-05, 3.16227766e-04,
       2.15443469e-03, 1.46779927e-02, 1.00000000e-01])

In [32]:
losses_tr, losses_val

(array([[ 0.69321697,  0.69109337,  0.66778917,  0.57948786,  1.34738355,
          1.08569301,  3.10374068],
        [ 4.03908174,  4.04052368,  4.05033894,  4.11682906,  4.55430702,
          4.08302902, 14.48442982],
        [12.51115184, 12.50823065, 12.48836791, 12.35465319, 11.51408103,
                 inf, 15.99151168],
        [13.2704321 , 13.26454842, 13.22456748, 12.95362378, 11.18296362,
          3.46197127,         inf],
        [        inf,         inf,         inf,         inf,         inf,
                 inf,         inf],
        [        inf,         inf,         inf,         inf,         inf,
         40.16907103, 11.31766831],
        [56.08533816, 56.05723225, 55.86618952, 54.58482614, 46.71767606,
         18.8172101 , 30.98938968]]),
 array([[nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
   

In [36]:
loss_min, lambda_star, gamma_star, max_accuracy

(nan, 0.014677992676220705, 0.1, 69.5592)

In [34]:
#PREDICT TRAINING ACCURACY
training_accuracy = predict_accuracy(y,tx,weights_star)
training_accuracy

59.4232

In [37]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [38]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [43]:
bound = [0.0, 1.0]
tx_test_clean,_ = treat_undefined_values (bound,tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [44]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [45]:
tx_test_augmented = build_poly_all_features(tx_test_clean,4)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre4.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)